# Logistic Regression from scratch in Python

While Python’s scikit-learn library provides the easy-to-use and efficient LogisticRegression class, the objective of this post is to create an own implementation using NumPy. Implementing basic models is a great idea to improve your comprehension about how they work.


Data set

We will use the well known Iris data set. It contains 3 classes of 50 instances each, where each class refers to a type of iris plant. To simplify things, we take just the first two feature columns. Also, the two non-linearly separable classes are labeled with the same category, ending up with a binary classification problem.

In [15]:
import sklearn.datasets
import numpy as np

In [3]:
iris = sklearn.datasets.load_iris()

In [5]:
X = iris.data[:, :2]
X

array([[5.1, 3.5],
       [4.9, 3. ],
       [4.7, 3.2],
       [4.6, 3.1],
       [5. , 3.6],
       [5.4, 3.9],
       [4.6, 3.4],
       [5. , 3.4],
       [4.4, 2.9],
       [4.9, 3.1],
       [5.4, 3.7],
       [4.8, 3.4],
       [4.8, 3. ],
       [4.3, 3. ],
       [5.8, 4. ],
       [5.7, 4.4],
       [5.4, 3.9],
       [5.1, 3.5],
       [5.7, 3.8],
       [5.1, 3.8],
       [5.4, 3.4],
       [5.1, 3.7],
       [4.6, 3.6],
       [5.1, 3.3],
       [4.8, 3.4],
       [5. , 3. ],
       [5. , 3.4],
       [5.2, 3.5],
       [5.2, 3.4],
       [4.7, 3.2],
       [4.8, 3.1],
       [5.4, 3.4],
       [5.2, 4.1],
       [5.5, 4.2],
       [4.9, 3.1],
       [5. , 3.2],
       [5.5, 3.5],
       [4.9, 3.6],
       [4.4, 3. ],
       [5.1, 3.4],
       [5. , 3.5],
       [4.5, 2.3],
       [4.4, 3.2],
       [5. , 3.5],
       [5.1, 3.8],
       [4.8, 3. ],
       [5.1, 3.8],
       [4.6, 3.2],
       [5.3, 3.7],
       [5. , 3.3],
       [7. , 3.2],
       [6.4, 3.2],
       [6.9,

In [13]:
y = (iris.target != 0) * 1
y.size

150

## Algorithm

Given a set of inputs X, we want to assign them to one of two possible categories (0 or 1). Logistic regression models the probability that each input belongs to a particular category.
Hypothesis

A function takes inputs and returns outputs. To generate probabilities, logistic regression uses a function that gives outputs between 0 and 1 for all values of X. There are many functions that meet this description, but the used in this case is the logistic function. From here we will refer to it as sigmoid.

$$
\begin{array} { l } { h _ { \theta } ( x ) = g \left( \theta ^ { T } x \right) } \\ { z = \theta ^ { T } x } \\ { g ( z ) = \frac { 1 } { 1 + e ^ { - z } } } \end{array}
$$

In [ ]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

z = np.dot(X, theta)
h = sigmoid(z)

## Loss function

Functions have parameters/weights (represented by theta in our notation) and we want to find the best values for them. To start we pick random values and we need a way to measure how well the algorithm performs using those random weights. That measure is computed using the loss function, defined as:

$$
\begin{array} { l } { h = g ( X \theta ) } \\ { J ( \theta ) = \frac { 1 } { m } \cdot \left( - y ^ { T } \log ( h ) - ( 1 - y ) ^ { T } \log ( 1 - h ) \right) } \end{array}
$$

In [17]:
def loss(h, y):
    return (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()

## Gradient descent

Our goal is to minimize the loss function and the way we have to achive it is by increasing/decreasing the weights, i.e. fitting them. The question is, how do we know what parameters should be biggers and what parameters should be smallers? The answer is given by the derivative of the loss function with respect to each weight. It tells us how loss would change if we modified the parameters.

$$
\frac { \delta J ( \theta ) } { \delta \theta _ { j } } = \frac { 1 } { m } X ^ { T } ( g ( X \theta ) - y )
$$

In [ ]:
gradient = np.dot(X.T, (h - y)) / y.shape[0]

Then we update the weights by substracting to them the derivative times the learning rate.

In [ ]:
lr = 0.01
theta -= lr * gradient

We should repeat this steps several times until we reach the optimal solution.

## Predictions

By calling the sigmoid function we get the probability that some input x belongs to class 1. Let’s take all probabilities ≥ 0.5 = class 1 and all probabilities < 0 = class 0. This threshold should be defined depending on the business problem we were working.

In [20]:
def predict_probs(X, theta):
    return sigmoid(np.dot(X, theta))

def predict(X, theta, threshold=0.5):
    return predict_probs(X, theta) >= threshold

## Putting it all together

In [35]:
class LogisticRegression:
    def __init__(self, lr=0.01, num_iter=100000, fit_intercept=True, verbose=False):
        self.lr = lr
        self.num_iter = num_iter
        self.fit_intercept = fit_intercept
    
    def __add_intercept(self, X):
        intercept = np.ones((X.shape[0], 1))
        return np.concatenate((intercept, X), axis=1)
    
    def __sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def __loss(self, h, y):
        return (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()
    
    def fit(self, X, y):
        if self.fit_intercept:
            X = self.__add_intercept(X)
        
        # weights initialization
        self.theta = np.zeros(X.shape[1])
        
        for i in range(self.num_iter):
            z = np.dot(X, self.theta)
            h = self.__sigmoid(z)
            gradient = np.dot(X.T, (h - y)) / y.size
            self.theta -= self.lr * gradient
            
            if(i % 10000 == 0):
                z = np.dot(X, self.theta)
                h = self.__sigmoid(z)
                print(f'loss: {self.__loss(h, y)} \t')
    
    def predict_prob(self, X):
        if self.fit_intercept:
            X = self.__add_intercept(X)
    
        return self.__sigmoid(np.dot(X, self.theta))
    
    def predict(self, X, threshold=0.5):
        return self.predict_prob(X) >= threshold

## Evaluation

In [36]:
model = LogisticRegression(lr=0.1, num_iter=300000)
%time model.fit(X, y)

loss: 0.6106904453410645 	
loss: 0.03432718644226483 	
loss: 0.02878665213455816 	
loss: 0.025718548517683616 	
loss: 0.02340842758463659 	
loss: 0.021507024522720526 	
loss: 0.019892368181604822 	
loss: 0.0185031664409479 	
loss: 0.017299193592041292 	
loss: 0.016249738703587754 	
loss: 0.015329838490314577 	
loss: 0.014518833591009722 	
loss: 0.013799605037762784 	
loss: 0.013158006956155629 	
loss: 0.012582374964505384 	
loss: 0.012063092454345654 	
loss: 0.011592216936374905 	
loss: 0.01116316678090668 	
loss: 0.010770464048285664 	
loss: 0.01040952627650975 	
loss: 0.01007649925054516 	
loss: 0.009768123172684362 	
loss: 0.009481625616205877 	
loss: 0.009214635760287644 	
loss: 0.008965115461932606 	
loss: 0.008731303635409655 	
loss: 0.008511671162408274 	
loss: 0.008304884158053572 	
loss: 0.008109773891466268 	
loss: 0.007925312028698209 	
CPU times: user 3.65 s, sys: 27.6 ms, total: 3.68 s
Wall time: 3.67 s


In [38]:
preds = model.predict(X)
preds

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [39]:
# accuracy
(preds == y).mean()
1.0

1.0

Picking a learning rate = 0.1 and number of iterations = 300000 the algorithm classified all instances successfully. 13.8 seconds were needed. These are the resulting weights:

In [40]:
model.theta

array([-25.89066442,  12.523156  , -13.40150447])